In [3]:
from keras.models import Sequential
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import style
from sklearn import svm, datasets
from sklearn.model_selection import cross_val_score
style.use("ggplot")

Using TensorFlow backend.


In [4]:
file = pd.read_csv("/srv/Program-pre/pyKMLib/examples/p4-1000000-2.csv")

Attribute (26 features)

- Ether_or_Dot3 : A packet is ethernet type or IEEE 802.3 type
- MAC_src : MAC address of source device
- MAC_dst : MAC address of destination device
- Ether_type: The type of ethernet packet, such as 0x0800 is Internet Protocol version 4 (IPv4)
- LLC : A packet is logical link control (LLC) type or not
- LLC_ssap : Source SAP field represents the logical address of LLC packet
- LLC_dsap : Destination SAP field represents the logical address of LLC packet
- IP_ttl : Time to live of a packet
- IP_version : The version of IP, such as IPv4
- TCP : A packet is TCP type
- UDP : A packet is UDP type
- ARP : A packet is ARP type
- ICMP : A packet is ICMP type
- pLen : A packet size
- Status : The network situation is attacked by DDoS or not
- num_ip_pair : The number of the pair of source ip and destination ip
- num_all_packets : Total of packets arrival per a window
- ratio_ip : Number of ip source divided by number of ip destination
- num_ip_src : Number of ip source
- num_ip_dst : Number of ip destination
- num_port_pair : The number of the pair of source port and destination port

In [5]:
file.head()

,Status,Ether_or_Dot3,MAC_src,MAC_dst,Ether_type,LLC,LLC_ssap,LLC_dsap,IP_ttl,IP_version,...,Num_all_packets,Ratio_ip,Num_ip_src,Num_ip_dst,Num_port_pair,Ratio_port,Num_port_src,Num_port_dst,Weight_ip,Weight_port
0,1,1,0.0,0.0,1.0,1.0,0.0,0.0,61,1.0,...,9715,0.003500,34,34,9,0.000926,5657,9,1.00,1.00
1,1,1,0.0,1.0,1.0,1.0,0.0,0.0,63,1.0,...,8725,0.012034,105,105,74,0.008481,74,3373,1.00,1.00
2,0,1,0.0,0.0,1.0,1.0,0.0,0.0,61,1.0,...,21338,0.040022,854,854,854,0.040022,12453,854,6.54,6.54
3,0,1,0.0,0.0,1.0,1.0,0.0,0.0,61,1.0,...,20965,0.003196,67,67,59,0.002814,12234,59,1.00,1.00
4,0,1,0.0,0.0,1.0,1.0,0.0,0.0,61,1.0,...,22183,0.005319,118,118,118,0.005319,13248,118,1.00,1.00


In [6]:
file['Status'].value_counts()

0    518097
1    481903
Name: Status, dtype: int64

In [8]:
x_train = file.drop('Status', axis=1)
x_train = x_train.values
y_train = file['Status'].values 

In [9]:
import time
from keras.layers import Dense, Dropout, ThresholdedReLU, Lambda
from keras.utils import CustomObjectScope
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=10)
cvscores = []

for train, test in kfold.split(x_train, y_train):
    start = time.time()

    model = Sequential()
    model.add(Dense(100, input_dim=25,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=50, activation='sigmoid'))
    model.add(Dropout(0.1))
    model.add(Dense(units=20, activation='sigmoid'))
    model.add(Dense(units=1, activation='sigmoid'))
    
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.fit(x_train[train], y_train[train], epochs=100, batch_size=10000)
    end = time.time()
    print "train time: "+str(end-start)

    start_r = time.time()
    y_pred = model.predict_classes(x_train[test],verbose=1)
    end_r = time.time()
    print("test time: "+str(end_r-start_r))
  
    a = mean_squared_error(y_train[test], y_pred)
    b = confusion_matrix(y_train[test], y_pred)
    print 'MES= ',a
    print 'Confusion matrix= ',b
    
    scores = model.evaluate(x_train[test], y_train[test], verbose=0)
    print model.metrics_names[1]
    print "metrics: "+str(model.metrics_names[1])+" score: "+str(scores[1]*100)
    cvscores.append(scores[1] * 100)


Epoch 1/100
899999/899999 [==============================] - 4s 4us/step - loss: 0.6201 - acc: 0.6539
Epoch 2/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.4318 - acc: 0.8016
Epoch 3/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.3545 - acc: 0.8443
Epoch 4/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.2941 - acc: 0.8799
Epoch 5/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.2233 - acc: 0.9157
Epoch 6/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.1610 - acc: 0.9429
Epoch 7/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.1229 - acc: 0.9568
Epoch 8/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0986 - acc: 0.9651
Epoch 9/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0840 - acc: 0.9706
Epoch 10/100
899999/899999 [==============================] - 2s 3us/step - loss: 

899999/899999 [==============================] - 2s 3us/step - loss: 0.0140 - acc: 0.9957
Epoch 81/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0150 - acc: 0.9954
Epoch 82/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0141 - acc: 0.9956
Epoch 83/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0136 - acc: 0.9959
Epoch 84/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0165 - acc: 0.9949
Epoch 85/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0156 - acc: 0.9952
Epoch 86/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0140 - acc: 0.9957
Epoch 87/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0141 - acc: 0.9956
Epoch 88/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0141 - acc: 0.9957
Epoch 89/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.01

899999/899999 [==============================] - 2s 3us/step - loss: 0.0192 - acc: 0.9942
Epoch 58/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0187 - acc: 0.9944
Epoch 59/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0175 - acc: 0.9948
Epoch 60/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0185 - acc: 0.9944
Epoch 61/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0181 - acc: 0.9946
Epoch 62/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0171 - acc: 0.9948
Epoch 63/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0175 - acc: 0.9948
Epoch 64/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0164 - acc: 0.9952
Epoch 65/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0168 - acc: 0.9948
Epoch 66/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.01

899999/899999 [==============================] - 2s 3us/step - loss: 0.0201 - acc: 0.9938
Epoch 35/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0190 - acc: 0.9943
Epoch 36/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0185 - acc: 0.9945
Epoch 37/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0212 - acc: 0.9935
Epoch 38/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0202 - acc: 0.9938
Epoch 39/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0194 - acc: 0.9940
Epoch 40/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0178 - acc: 0.9948
Epoch 41/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0191 - acc: 0.9944
Epoch 42/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.0183 - acc: 0.9947
Epoch 43/100
899999/899999 [==============================] - 2s 3us/step - loss: 0.01

900000/900000 [==============================] - 2s 3us/step - loss: 0.0560 - acc: 0.9818
Epoch 12/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0514 - acc: 0.9834
Epoch 13/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0466 - acc: 0.9848
Epoch 14/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0430 - acc: 0.9859
Epoch 15/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0365 - acc: 0.9879
Epoch 16/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0348 - acc: 0.9881
Epoch 17/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0324 - acc: 0.9889
Epoch 18/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0315 - acc: 0.9891
Epoch 19/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0284 - acc: 0.9904
Epoch 20/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.02

...

900000/900000 [==============================] - 2s 3us/step - loss: 0.0354 - acc: 0.9890
Epoch 22/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0314 - acc: 0.9905
Epoch 23/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0298 - acc: 0.9911
Epoch 24/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0288 - acc: 0.9915
Epoch 25/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0299 - acc: 0.9911
Epoch 26/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0283 - acc: 0.9915
Epoch 27/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0322 - acc: 0.9904
Epoch 28/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0264 - acc: 0.9924
Epoch 29/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.0306 - acc: 0.9909
Epoch 30/100
900000/900000 [==============================] - 2s 3us/step - loss: 0.02

900000/900000 [==============================] - 2s 3us/step - loss: 0.0165 - acc: 0.9951
train time: 238.549851894
100000/100000 [==============================] - 15s 146us/step
test time: 14.6486270428
MES=  0.00327
Confusion matrix=  [[51778    32]
 [  295 47895]]
acc
metrics: acc score: 99.673
Epoch 1/100
900001/900001 [==============================] - 3s 3us/step - loss: 0.6086 - acc: 0.6766
Epoch 2/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.4063 - acc: 0.8267
Epoch 3/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.2970 - acc: 0.8864
Epoch 4/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.2947 - acc: 0.8814
Epoch 5/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.2339 - acc: 0.9105
Epoch 6/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.1784 - acc: 0.9349
Epoch 7/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.1437 

900001/900001 [==============================] - 2s 3us/step - loss: 0.0222 - acc: 0.9936
Epoch 78/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0225 - acc: 0.9936
Epoch 79/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0230 - acc: 0.9933
Epoch 80/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0212 - acc: 0.9939
Epoch 81/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0232 - acc: 0.9931
Epoch 82/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0244 - acc: 0.9928
Epoch 83/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0216 - acc: 0.9937
Epoch 84/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0208 - acc: 0.9940
Epoch 85/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0215 - acc: 0.9938
Epoch 86/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.02

900001/900001 [==============================] - 2s 3us/step - loss: 0.0212 - acc: 0.9942
Epoch 55/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0223 - acc: 0.9938
Epoch 56/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0214 - acc: 0.9941
Epoch 57/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0224 - acc: 0.9936
Epoch 58/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0411 - acc: 0.9882
Epoch 59/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0287 - acc: 0.9916
Epoch 60/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0288 - acc: 0.9916
Epoch 61/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0263 - acc: 0.9924
Epoch 62/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0271 - acc: 0.9920
Epoch 63/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.03

900001/900001 [==============================] - 2s 3us/step - loss: 0.0354 - acc: 0.9878
Epoch 32/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0340 - acc: 0.9886
Epoch 33/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0329 - acc: 0.9886
Epoch 34/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0316 - acc: 0.9892
Epoch 35/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0301 - acc: 0.9898
Epoch 36/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0293 - acc: 0.9902
Epoch 37/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0277 - acc: 0.9910
Epoch 38/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0290 - acc: 0.9904
Epoch 39/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.0286 - acc: 0.9908
Epoch 40/100
900001/900001 [==============================] - 2s 3us/step - loss: 0.02

In [10]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

99.63% (+/- 0.06%)


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 100)               2600      
_________________________________________________________________
dropout_19 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_20 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 21        
Total params: 8,691
Trainable params: 8,691
Non-trainable params: 0
_________________________________________________________________


Save model

In [10]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model-maxp-dff.json","w") as json_file:
    json_file.write(model_json)
model.save_weights("model-maxp-dff.h5")